# Running models and cross validation 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from termcolor import colored
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold
from sklearn import metrics

C:\Users\luisf\AppData\Local\Temp\ipykernel_2792\4088898045.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
#Time series split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA

In [2]:
# Loading final dataset
df = pd.read_csv('dummies_cleaned.csv')
print(df.head())
print(df.shape)

#Counting NA
nan_count = df['energy_price'].isna().sum()
print(nan_count)

#df_cleared = df.dropna(subset=['energy_price'])
#print(df_cleared.shape)

   Unnamed: 0  energy_price     biomass          gas      nuclear  year  \
0           0         44.94  252.200902  1180.283774  7185.089448  2015   
1           1         26.48  135.000000   172.000000  2596.000000  2015   
2           2         25.02   18.000000   233.000000  7185.089448  2015   
3           3         27.38   25.000000   304.000000  7185.089448  2015   
4           4         27.38  252.200902  1180.283774  7185.089448  2015   

   month  day  hour  week_number  ...  week_number_51  week_number_52  \
0      1    1     0            1  ...               0               0   
1      1    1     0            1  ...               0               0   
2      1    1     0            1  ...               0               0   
3      1    1     0            1  ...               0               0   
4      1    1     0            1  ...               0               0   

   week_number_53  day_of_week_0  day_of_week_1  day_of_week_2  day_of_week_3  \
0               0            

In [3]:
# Separating features and target variable
y = df['energy_price']
X = df.drop(['energy_price'], axis=1)

In [6]:
#Train-test split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Splitting the data
split_point = int(len(y) * 0.8)
y_train, y_test = y[0:split_point], y[split_point:]
X_train, X_test = X[0:split_point], X[split_point:]
df_train, df_test = df_cleared[0:split_point], df_cleared[split_point:]
print(y_train)

1976013     44.94
1976014     26.48
1976017     25.02
1976018     27.38
1976019     27.38
            ...  
4838646      9.45
4838647    202.81
4838648     56.96
4838649    -30.00
4838650    138.05
Name: energy_price, Length: 2697072, dtype: float64


In [7]:
# Fit model
model = ARIMA(y_train, order=(1,0,0))  # example order, adjust based on your data
fitted_model = model.fit()


# Forecast
predictions = fitted_model.forecast(steps=len(y_test))



c:\Users\cpedr\miniconda3\envs\ml_lab\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\cpedr\miniconda3\envs\ml_lab\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\cpedr\miniconda3\envs\ml_lab\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\cpedr\miniconda3\envs\ml_lab\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\cpedr\miniconda3\envs\ml_lab\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: F

In [14]:
# Evaluate the model
mse = mean_squared_error(y_test, predictions)
print(f'MSE: {mse}')

# Plot
plt.figure(figsize=(10, 6))
plt.plot(test.index, test, label='Actual')
plt.plot(test.index, predictions, label='Forecasted', color='red')
plt.legend()
plt.show()

print(predictions)


ValueError: Input contains NaN.

In [10]:
print(fitted_model)

Basic linear model with latest dataset ("dummies_cleaned.csv")

In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv("dummies_cleaned.csv")

In [4]:
y = df['energy_price']
X = df.drop(['energy_price'], axis=1)


In [5]:
print(y.describe)
print(X.info)

<bound method NDFrame.describe of 0          44.94
1          26.48
2          25.02
3          27.38
4          27.38
           ...  
3294775    44.87
3294776    44.87
3294777    44.87
3294778    33.30
3294779    35.14
Name: energy_price, Length: 3294780, dtype: float64>
<bound method DataFrame.info of          Unnamed: 0                 Date Country     biomass          gas  \
0                 0  2015-01-01 00:00:00      CH  252.200902  1180.283774   
1                 1  2015-01-01 00:00:00      CZ  135.000000   172.000000   
2                 2  2015-01-01 00:00:00    DK_1   18.000000   233.000000   
3                 3  2015-01-01 00:00:00    DK_2   25.000000   304.000000   
4                 4  2015-01-01 00:00:00      EE  252.200902  1180.283774   
...             ...                  ...     ...         ...          ...   
3294775     3294775  2023-12-31 00:00:00    SE_2  252.200902  1180.283774   
3294776     3294776  2023-12-31 00:00:00    SE_3  252.200902  1180.283774   
3

In [8]:
#splitting set into training and data for this naive linear model
seed = 42

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = seed)

print(X.shape)
print(y.shape)

(3294780, 199)
(3294780,)


In [11]:
model = LinearRegression()
model.fit(X_train, y_train)

MemoryError: Unable to allocate 3.91 GiB for an array with shape (199, 2635824) and data type object

In [ ]:
y_pred = model.predict(X_test)

# Calculating MSE
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.3)
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=4)
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Actual vs. Predicted')
plt.show()